In [2]:
import sgml, sgutil, sgpp, dproc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import root_mean_squared_error

from proc_v2 import p
from ml_v1 import X_cat, X_num, X_all, target, kf, ss, config
from ml_v1 import xgb_adapter, lgb_adapter, cb_adapter, lr_adapter, nn_adapter

sc = sgutil.SGCache('img', 'result', 'model')
df_train = p.fit_transform(['data/train.csv']).assign(
    Calories_Log = lambda x: np.log(x['Calories'] + 1)
)
df_test = p.transform(['data/test.csv'])

2025-05-20 01:31:13.274010: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-20 01:31:13.285114: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747672273.298482    3743 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747672273.303244    3743 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747672273.313840    3743 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
df_train['Duration'].nunique()

30

In [4]:
df_train['Heart_Rate'].nunique()

63

In [5]:
df_train['Body_Temp'].nunique()

75

# Target Encoder의 효과를 봅니다.

In [23]:
df_train.groupby(['Duration', 'Heart_Rate', 'Body_Temp'])[target].agg(
    ['mean', 'std', 'size']
).query('size >= 50')['std'].mean()

0.12801187

In [14]:
df_train.groupby(['Duration', 'Heart_Rate'])[target].agg(
    ['mean', 'std', 'size']
).query('size >= 50')['std'].mean()

0.14161234

In [15]:
df_train.groupby(['Duration', 'Body_Temp'])[target].agg(
    ['mean', 'std', 'size']
).query('size >= 50')['std'].mean()

0.1881298

In [16]:
df_train.groupby(['Duration'])[target].agg(
    ['mean', 'std', 'size']
).query('size >= 50')['std'].mean()

0.17675276

In [24]:
df_train.groupby(['Sex', 'Duration', 'Heart_Rate', 'Body_Temp'])[target].agg(
    ['mean', 'std', 'size']
).query('size >= 50')['std'].mean()

0.10964416

In [19]:
df_train.groupby(['Height', 'Weight'])[target].agg(
    ['mean', 'std', 'size']
).query('size >= 50')['std'].mean()

0.9575898

In [20]:
df_train.groupby(['Height'])[target].agg(
    ['mean', 'std', 'size']
).query('size >= 50')['std'].mean()

0.9839047

In [21]:
df_train.groupby(['Weight'])[target].agg(
    ['mean', 'std', 'size']
).query('size >= 50')['std'].mean()

1.0028571

In [25]:
df_train.groupby(['Sex', 'Height', 'Weight'])[target].agg(
    ['mean', 'std', 'size']
).query('size >= 50')['std'].mean()

0.9559728

In [28]:
cc = sgpp.CatCombiner2(
    [['Sex', 'Duration', 'Heart_Rate', 'Body_Temp'], ['Sex', 'Height', 'Weight']]
)
df_train = dproc.join_and_assign(
    df_train, cc.fit_transform(df_train)
)

In [51]:
sgml.cv(df_train, kf, {'X_num': ['Duration_log', 'Age', 'Heart_Rate_sqrt_d'], 
                       'X_tgt': df_train.columns[-2:].tolist(), 'tgt': {'smooth': 0.1}}, config, lr_adapter)

Fold:   0%|          | 0/4 [00:00<?, ?it/s]

{'valid_scores': [0.09580501430537243,
  0.09554967334257566,
  0.0950815439172017,
  0.09612906550500196],
 'valid_prd': id
 0         4.969359
 1         3.597198
 2         3.367708
 3         4.912362
 4         4.977167
             ...   
 749995    5.349983
 749996    4.583249
 749997    5.483796
 749998    4.662099
 749999    4.596092
 Length: 750000, dtype: float64,
 'model_result': [],
 'train_scores': [0.09258123041642373,
  0.09261837002672306,
  0.09272315387427311,
  0.09250549421345512],
 'hparams': {'X_num': ['Duration_log', 'Age', 'Heart_Rate_sqrt_d'],
  'X_tgt': ['Sex__Duration__Heart_Rate__Body_Temp', 'Sex__Height__Weight'],
  'tgt': {'smooth': 0.1}}}

Index(['Sex__Duration__Heart_Rate__Body_Temp', 'Sex__Height__Weight'], dtype='object')